<h1>AUTOMATIZACIÓN CONSOLIDADO SAT PUSH EXPERIENCIA<h1>

In [1]:
import pandas as pd
import os
import glob
import re

In [2]:
mes_anterior_num = "05"
mes_anterior_str = "Mayo"

In [6]:
carpeta_excel = f'C:/Users/BBARTOLOMEE/OneDrive - Entel Peru S.A/03. Programación Diaria Comunicaciones/02. Bases Satpush (CVM)/2025/{mes_anterior_num}. {mes_anterior_str}'
archivos_excel = glob.glob(os.path.join(carpeta_excel, '*.xlsx'))

dataframes_excel = []

In [7]:
for archivo in archivos_excel:
    try:
        df = pd.read_excel(archivo)
        nombre_archivo = os.path.basename(archivo).replace('.xlsx', '')
        df['archivo'] = nombre_archivo
        dataframes_excel.append(df)
    except Exception as e:
        print(f"Error leyendo {archivo}: {e}")

df_consolidado = pd.concat(dataframes_excel, ignore_index=True)

In [8]:
df_consolidado

,TELEFONO,archivo
0,51908925923,20250508_CVM_SATPUSH_LLAA
1,51918844414,20250508_CVM_SATPUSH_LLAA
2,51922138624,20250508_CVM_SATPUSH_LLAA
3,51922909329,20250508_CVM_SATPUSH_LLAA
4,51923520019,20250508_CVM_SATPUSH_LLAA
...,...,...
23995,51986687176,20250508_CVM_SATPUSH_LLAA
23996,51970784568,20250508_CVM_SATPUSH_LLAA
23997,51926896927,20250508_CVM_SATPUSH_LLAA
23998,51956646448,20250508_CVM_SATPUSH_LLAA


<h3>BASE RUCS<h3>

In [ ]:
carpeta_rucs = f'C:/Users/BBARTOLOMEE/OneDrive - Entel Peru S.A/03. Programación Diaria Comunicaciones/02. Bases Satpush (CVM)/2025/RUC {mes_anterior_str}.xlsx'

In [10]:
df_base_ruc = pd.read_excel(carpeta_rucs)
df_base_ruc.drop_duplicates(subset='TELEFONO')

,TELEFONO,RUCCOMPANIA
0,51908925923,20147170212
1,51918844414,20601479126
2,51922138624,20602110231
3,51922909329,20601989060
4,51923520019,20600824571
...,...,...
49500,51986124896,20521603322
49501,51969197393,20392477986
49502,51964820899,20601281121
49503,51960313753,20609232545


In [11]:
df_merge = pd.merge(df_consolidado, df_base_ruc, on='TELEFONO', how='left')

In [12]:
df_merge['nombre_base'] = df_merge['archivo'].str.extract(r'_(?:.*?)_(.*)$')
df_merge['FECHA'] = pd.to_datetime(df_merge['archivo'].str[:8], format='%Y%m%d')
df_merge['FECHA'] = df_merge['FECHA'].dt.strftime('%d-%m-%Y')

In [13]:
df_merge

,TELEFONO,archivo,RUCCOMPANIA,nombre_base,FECHA
0,51908925923,20250508_CVM_SATPUSH_LLAA,20147170212,SATPUSH_LLAA,08-05-2025
1,51918844414,20250508_CVM_SATPUSH_LLAA,20601479126,SATPUSH_LLAA,08-05-2025
2,51922138624,20250508_CVM_SATPUSH_LLAA,20602110231,SATPUSH_LLAA,08-05-2025
3,51922909329,20250508_CVM_SATPUSH_LLAA,20601989060,SATPUSH_LLAA,08-05-2025
4,51923520019,20250508_CVM_SATPUSH_LLAA,20600824571,SATPUSH_LLAA,08-05-2025
...,...,...,...,...,...
24111,51986687176,20250508_CVM_SATPUSH_LLAA,20536729225,SATPUSH_LLAA,08-05-2025
24112,51970784568,20250508_CVM_SATPUSH_LLAA,20547687613,SATPUSH_LLAA,08-05-2025
24113,51926896927,20250508_CVM_SATPUSH_LLAA,20608389076,SATPUSH_LLAA,08-05-2025
24114,51956646448,20250508_CVM_SATPUSH_LLAA,20495084257,SATPUSH_LLAA,08-05-2025


In [ ]:
#df_nombre = df_merge[['nombre_base']].drop_duplicates()
#df_nombre.to_excel('base_campaña.xlsx',index = False)

<h3>BASE CAMPAÑAS<h3>

In [14]:
base_campaña = pd.read_excel('base_campaña.xlsx')
base_final_1 = pd.merge(df_merge, base_campaña, on = 'nombre_base', how='left')

In [15]:
base_final_1[['TIPO']] = 'SAT PUSH'
base_final_1[['OWNER']] = 'CVM'

In [16]:
base_final_2 = base_final_1[['FECHA','RUCCOMPANIA','TELEFONO','OWNER','Campaña','TIPO']]
base_final_2.to_excel('df_final.xlsx',index = False)